In [ ]:
%use plotly
import kotlinx.coroutines.*
import java.time.Duration
import java.time.Instant

val data = runBlocking { readAllUData("./data/log.anon") + readAllUData("./data/log2.anon") }
val cacheStats = parseQueryLoad("./data/Query-load.csv") + parseQueryLoad("./data/Query-load2.csv")

In [ ]:
fun add0WithinData(toFill: List<UData>): List<Pair<UInt, List<UData>>> {
    val filledQueryPerSecond = mutableListOf<Pair<UInt, List<UData>>>()
    var current = mutableListOf<UData>()
    var currentTime = toFill.first().unixTimeStamp
    toFill.forEach { u ->
        if (u.unixTimeStamp == currentTime) {
            current.add(u)
        } else {
            filledQueryPerSecond.add(Pair(currentTime, current))
            val afterFirst = currentTime + 1u
            val beforeLast = u.unixTimeStamp - 1u
            if (afterFirst == beforeLast) {
                filledQueryPerSecond.add(Pair(afterFirst, listOf()))
            } else if (afterFirst < beforeLast) {
                filledQueryPerSecond.add(Pair(afterFirst, listOf()))
                filledQueryPerSecond.add(Pair(beforeLast, listOf()))
            }
            current = mutableListOf(u)
            currentTime = u.unixTimeStamp
        }
    }
    filledQueryPerSecond.add(Pair(currentTime, current))
    return filledQueryPerSecond
}

val splitPerSuffix = data.groupBy { ud -> ud.domainName }.mapValues { (_, v) -> add0WithinData(v) }

val queryPerSecond = add0WithinData(data)

fun plotOverTime(
    udata: Map<List<String>, List<Pair<UInt, List<UData>>>>,
    title: String,
    includeSub: Boolean = true,
    cacheStats: List<CacheStat> = listOf(),
): Plot {
    val offset = Duration.ofHours(2)

    val perSuffixTraces = udata.map { (suffix, v) ->
        Scatter() {
            val values = v.map { (s, udl) -> Pair(s, udl.filter { ud -> includeSub || ud.sub == false }) }
            val yNumbers = values.map { (_, udl) -> udl.size }
            name = suffix.joinToString(".")
            x.strings = values.map { (t, _) ->
                Instant.ofEpochSecond(t.toLong()).plus(offset).toString() // Convert to CET
            }
            y.numbers = yNumbers
        }
    }.sortedByDescending { t -> t.y.numbers.sumOf { n -> n.toInt() } }

    val amountOfQueriesTrace = Scatter() {
        name = "Requests"
        x.strings = cacheStats.map { cs -> cs.time.toInstant().plus(offset).toString() }
        y.numbers = cacheStats.map { cs -> (cs.cacheHits + cs.cacheMisses) }
        yaxis = "y2"
        marker {
            color("rgba(44,160,44,1)")
        }
    }

    val cacheHitsTrace = Scatter() {
        name = "Cache hits"
        x.strings = cacheStats.map { cs -> cs.time.toInstant().plus(offset).toString() }
        y.numbers = cacheStats.map { cs -> cs.cacheHits }
        yaxis = "y2"
    }

    val cacheMisses = Scatter() {
        name = "Cache misses"
        x.strings = cacheStats.map { cs -> cs.time.toInstant().plus(offset).toString() }
        y.numbers = cacheStats.map { cs -> cs.cacheMisses }
        yaxis = "y2"
    }

    return Plotly.plot {
        traces(
            if (cacheStats.isEmpty()) {
                perSuffixTraces
            } else {
                perSuffixTraces + amountOfQueriesTrace
            }
        )

        layout {
            width = 1700 * 0.5
            height = 950 * 0.5
            title {
                text = "$title"
            }
            xaxis {
                title {
                    text = "Time"
                }
            }
            if (cacheStats.isEmpty()) {
                yaxis {
                    title {
                        text = "Amount of validations per second"
                    }
                    type = AxisType.linear
                }
            } else {
                yaxis {
                    title {
                        text = "Validations per second"
                    }
                    color("rgba(31,119,180,1")
                    type = AxisType.linear
                    overlaying = "y2"
                }
                yaxis(2, {
                    title {
                        text = "Requests per second"
                    }
                    side = AxisSide.right
                    color("rgba(44,160,44,1)")
                })
            }
        }
    }
}

In [ ]:
plotOverTime(
    mapOf(Pair(listOf("Validations"), queryPerSecond)),
    "Amount of requests and signature validations over time",
    cacheStats = cacheStats
).apply {
    layout {
        showlegend = false
        legend {
            traceorder = TraceOrder.reversed
        }
    }
}

In [ ]:
plotOverTime(mapOf(Pair(listOf("all"), queryPerSecond)), "Amount of validations over time")

In [ ]:
val amountRootZone =
    queryPerSecond.map { (time, data) -> Pair(time, data.filter { ud -> ud.domainName.size == 1 }) }

plotOverTime(mapOf(Pair(listOf(), amountRootZone)), "Amount of validations over time for root zone")

In [ ]:
val amountTLDs =
    queryPerSecond.map { (time, data) -> Pair(time, data.filter { ud -> ud.domainName.size <= 2 }) }

plotOverTime(
    mapOf(Pair(listOf("TLDs and root zone"), amountTLDs)),
    "Amount of validations over time for TLDs and root zone",
    false
)

In [ ]:
plotOverTime(splitPerSuffix, "Amount of validations over time per suffix including subdomains")

In [ ]:
plotOverTime(
    splitPerSuffix,
    "Amount of validations over time per suffix excluding subdomains",
    false
)